# --

In [37]:
# https://habr.com/ru/post/276725/
# способ создания красивых прогресс-баров
# см. также pypi.python.org/pypi/tqdm
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [46]:
# прогресс-бар числом - для случаев while и т.п.
from ipywidgets import HTML
from IPython.display import display
from time import sleep
label = HTML()
display(label)
for x in range(10):
    label.value = str(x)
    sleep(0.1)

HTML(value='')

In [59]:
# вычисляет полный размер объекта в памяти

import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType


def getmemsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

In [2]:
# проверка на симлинки
# стандартная проверка в windows junction-ы не считает симлинками

from ctypes import *
from ctypes.wintypes import *

FILE_ATTRIBUTE_REPARSE_POINT = 0x00400
INVALID_FILE_ATTRIBUTES = 0xFFFFFFFF

kernel32 = WinDLL('kernel32')
GetFileAttributesW = kernel32.GetFileAttributesW
GetFileAttributesW.restype = DWORD
GetFileAttributesW.argtypes = (LPCWSTR,) #lpFileName In

def islink(path):
    result = GetFileAttributesW(path)
    if result == INVALID_FILE_ATTRIBUTES:
        raise WinError()
    return bool(result & FILE_ATTRIBUTE_REPARSE_POINT)

islink(r'D:\Users\feelus\Local Settings')

True

In [56]:
# сканирует заданную папку и возвращает дерево:
# имя_папки : {...}
# имя_файла : [размер]

import os
from stat import *

def scan(rootpath):
    root = {}
    #rootpath = 'D:\\'
    total_size = 0
    ts_printed = 0

    label = HTML()
    display(label)
    
    for curdir, indirs, infiles in os.walk(rootpath):
        path = curdir
        folders = []
        while path != rootpath:
            #print(path)
            path, folder = os.path.split(path)
            folders.append(folder)

        #print(curdir)
        #print(folders)
        curroot = root
        try:
            for i in reversed(folders):
                curroot = curroot[i]
        except:
            #print('path is apsent in tree:',curdir)
            continue

        for d in indirs:
            cur_d = os.path.join(curdir,d)
            try:
                mode = os.stat(cur_d).st_mode
                islnk = islink(cur_d)
            except:
                print("can't check stat of dir:",cur_d)
                continue
            if not S_ISDIR(mode) or islnk:
                print("this dir is not dir:",cur_d,mode)
                continue
            curroot[d]={}
        for f in infiles:
            cur_f = os.path.join(curdir,f)
            try:
                st = os.stat(cur_f)
                mode = st.st_mode
                size = st.st_size
                islnk = islink(cur_d)
            except:
                print("can't check stat of file:",cur_f)
                continue
            if not S_ISREG(mode):
                print("this file is not file:",cur_f,mode)
                continue
            if islnk:
                print("this file is not usual:",cur_f,mode)
            curroot[f]=[size]
            
            total_size+=size
            if ts_printed<int(total_size/1024/1024/1024):
                ts_printed = int(total_size/1024/1024/1024)
                label.value = str(ts_printed)+' GB scanned'
            
    label.value = str(ts_printed)+' GB scanned - completed'
    return root


In [48]:
# проходится по всему дереву
# на каждом узле(листе) возвращает пару (путь, значение)
# где путь - список имен, по которым надо добираться по дереву до значения

def tree_iterator(tree):
    for k,v in tree.items():
        if type(v)==dict:
            for path,v2 in tree_iterator(v):
                path.insert(0,k)
                yield path,v2
        else:
            yield [k],v

In [6]:
r = {'a':1,'b':{'c':2,'d':3}}
for x in tree_iterator(r):
    print(x)

(['a'], 1)
(['b', 'c'], 2)
(['b', 'd'], 3)


In [49]:
# берет корень и путь, проходит по пути, и возвращает то, где оказался

def get_subtree(root,path):
    if len(path)==0: return root
    else: return get_subtree(root[path[0]],path[1:])

In [50]:
# вычисляет хеш файла по его пути

import hashlib
def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [67]:
def get_diplics_size(root_YD):
    size_root_YD={} # map размер -> множество файлов
    for x,v in tree_iterator(root_YD):
        size = v[0]
        file = tuple(x)
        #print(x,v,size)
        if size in size_root_YD:
            size_root_YD[size].add(file)
        else:
            size_root_YD[size] = {file}

    # map размер -> кол-во повторений
    duplics1_YD = [(i,len(size_root_YD[i])) for i in reversed(sorted(size_root_YD.keys())) \
                   if len(size_root_YD[i])!=1]
    tot_s=0
    tot_ss = 0
    tot_n = 0
    for s,n in duplics1_YD:
        tot_s+=s*(n-1)
        tot_ss+=s*(n)
        tot_n+=n
    print('размер повторений', '%.3f'%(tot_ss/1024/1024),'MB')
    print('размер лишнего   ', '%.3f'%(tot_s/1024/1024),'MB')
    print('кол-во разных    ', len(duplics1_YD))
    print('кол-во повторений', tot_n)
    return size_root_YD,duplics1_YD


In [69]:
def list_duplics_size(size_root_YD,duplics1_YD):
    for s,n in duplics1_YD:
        for p in sorted(size_root_YD[s]):
            yield s,p

# YandexDisk

In [57]:
rootpath_YD = r'D:\Users\feelus\YandexDisk'
root_YD = scan(rootpath_YD)
len(root_YD)

HTML(value='')

can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_data acces and storage\z_Local File Systems\z_Directory Management\About Directory Management (Windows)_files\030c41d9079671d09a62d8e2c1db6973.gif
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_data acces and storage\z_Local File Systems\z_Directory Management\About Directory Management (Windows)_files\0d0542f5f45cf9fc7273abd11cf5c0ee.gif
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_data acces and storage\z_Local File Systems\z_Directory Management\About Directory Management (Windows)_files\8e010b67ceb176b577f5afa1ceb6f5ad.css
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows devel

can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_data acces and storage\z_Local File Systems\z_File Management\z_About File Management\Files and Clusters (Windows)_files\c764fa3d6b8a84bd40aa131355215c98.png
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_data acces and storage\z_Local File Systems\z_File Management\z_About File Management\Reading From and Writing to Files (Windows)_files\Combined.css
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_data acces and storage\z_Local File Systems\z_File Management\z_About File Management\Reading From and Writing to Files (Windows)_files\Sync.htm
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_wi

can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_getting started\z_learn to program  for windows in c++\Module 1. Your First Windows Program (Windows)_files\0d0542f5f45cf9fc7273abd11cf5c0ee.gif
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_getting started\z_learn to program  for windows in c++\Module 1. Your First Windows Program (Windows)_files\8e010b67ceb176b577f5afa1ceb6f5ad.css
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_getting started\z_learn to program  for windows in c++\Module 1. Your First Windows Program (Windows)_files\c764fa3d6b8a84bd40aa131355215c98.png
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development

can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\_msdn_html\z_windows development\z_getting started\z_learn to program  for windows in c++\z_Module 3. Windows Graphics\Drawing with Direct2D (Windows)_files\030c41d9079671d09a62d8e2c1db6973.gif
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\_msdn_html\z_windows development\z_getting started\z_learn to program  for windows in c++\z_Module 3. Windows Graphics\Drawing with Direct2D (Windows)_files\0d0542f5f45cf9fc7273abd11cf5c0ee.gif
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\_msdn_html\z_windows development\z_getting started\z_learn to program  for windows in c++\z_Module 3. Windows Graphics\Drawing with Direct2D (Windows)_files\8e010b67ceb176b577f5afa1ceb6f5ad.css
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\_msdn_html

26

In [60]:
getmemsize(root_YD)/1024/1024

11.627914428710938

In [68]:
size_root_YD,duplics1_YD = get_diplics_size(root_YD)

размер повторений 2215.181 MB
размер лишнего    1299.934 MB
кол-во разных     9989
кол-во повторений 54421


In [72]:
tot_ss = 0
tot_n = 0

label = HTML()
display(label)

for s,p in list_duplics_size(size_root_YD,duplics1_YD):
    hashh = md5(os.path.join(rootpath_YD,*p))
    fs = get_subtree(root_YD,p)
    if len(fs) == 1: fs.append(hashh)
    else:            fs[1] = hashh
    #print(hashh+'\t'+os.path.join(rootpath_YD,*p))
    tot_ss += s
    tot_n+=1
    label.value = str(tot_n)+' files,  ' +'%.3f'%(tot_ss/1024/1024)+' MB'

HTML(value='')

In [74]:
duplics2_YD = {}
for s,n in duplics1_YD:
    for p in sorted(size_root_YD[s]):
        fdata = get_subtree(root_YD,p)
        assert len(fdata)>=2, (p,fdata)
        if len(fdata)>2: del fd[2:]
        hashh = fdata[1]
        if (s,hashh) in duplics2_YD:
            duplics2_YD[(s,hashh)].add(p)
        else:
            duplics2_YD[(s,hashh)] = {p}
duplics2_YD = {s_h:pp for (s_h,pp) in duplics2_YD.items() if len(pp)>1}
tot_s=0
tot_ss = 0
tot_n = 0
for s_h,pp in duplics2_YD.items():
    n = len(pp)
    s = s_h[0]
    tot_s+=s*(n-1)
    tot_ss+=s*(n)
    tot_n+=n
print('размер всего, размер лишнего, кол-во разных, кол-во всего')
print(tot_ss/1024/1024,tot_s/1024/1024,len(duplics2_YD),tot_n)

размер всего, размер лишнего, кол-во разных, кол-во всего
1353.1258811950684 762.9555749893188 12623 34688


In [25]:
for s_h,pp in [(a,b) for a,b in duplics2_YD.items()]:
    s,h = s_h
    for p in pp:
        if p[0]=='_programming' and p[1]=='_lang_cpp' :
            print('%.3f'%(s/1024),h)
            for q in pp:
                print('\t'+os.path.join(rootpath_YD,*q))
            break

1104.473 09ef7471245e968d41023d8f5d2fa693
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\грамматика с++\грамматика с++\file_types\__grammar__.graphml
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\грамматика с++\грамматика с++\handmade\__grammar__.graphml
760.053 cfd0876bf91863fdf4a33b8bb17cc45f
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\грамматика с++\c++_handmade_1_4_OR_inet.graphml
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\грамматика с++\c++98_handmade_4_1.graphml
302.051 5b2e7f988f3955df16584f09ea82ad07
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\.files\2017-07-19 19-58-29-files
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\.files\2017-07-22 13-00-56-files
256.000 a6ba93fee5eea07697a05ebaf5cd906a
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\приложения\BC_3.1\BC\TC0002.SWP
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\приложения\BC_3.1\BC\TC0001.SWP
256.000 2b9bd1283dbe5306d7a08778469f4cc2
	D:\Users\feelus\YandexDisk\_programming\_lang

	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\msdn_html-куски\z_windows development\z_data acces and storage\z_Local File Systems\z_File Management\z_About File Management\File and Directory Linking (Windows)_files\Sync.htm
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\_msdn_html\z_windows development\z_data acces and storage\z_Local File Systems\z_File Management\z_About File Management\Sparse Files (Windows)_files\Sync.htm
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\_msdn_html\z_windows development\z_data acces and storage\z_Local File Systems\z_Disk Management\Disk Management Reference (Windows)_files\Sync.htm
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn _html\_msdn_html\z_windows development\z_data acces and storage\z_Local File Systems\z_Disk Management\About Disk Management (Windows)_files\Sync.htm
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов

In [36]:
for s_h,pp in [(a,b) for a,b in duplics2.items()]:
    s,h = s_h
    for p in pp:
        if p[0]=='multimedia':
            print(int(s),h)
            for p in pp:
                print('\t'+os.path.join(rootpath,*p))
            break

7569 0a025414a5f4cfc778f8a52ef5d2ffb7
	D:\Users\feelus\YandexDisk\multimedia\.files\2017-07-08 16-30-48-dirs
	D:\Users\feelus\YandexDisk\multimedia\.files\2017-07-08 16-27-57-dirs
325 21e67c73585028975d7ab33cc808ed94
	D:\Users\feelus\YandexDisk\multimedia\статы - Видео Шведова\content-link.txt
	D:\Users\feelus\YandexDisk\_физика\5_статистическая физика\content-link.txt
212 cb0de00f716742da5d504fe8e181062e
	D:\Users\feelus\YandexDisk\окюу3\desktop.ini
	D:\Users\feelus\YandexDisk\multimedia\audio\desktop.ini
	D:\Users\feelus\YandexDisk\_физика\3_квантовые вычисления\desktop.ini
	D:\Users\feelus\YandexDisk\_programming_arxiv\lang_asm\desktop.ini
	D:\Users\feelus\YandexDisk\multimedia\фото - 409-м\desktop.ini
	D:\Users\feelus\YandexDisk\программы-установщики\_крякнутые\ABBYY Screenshot Reader 11.0.113.201 RePack by KpoJIuK\desktop.ini
	D:\Users\feelus\YandexDisk\multimedia\обои рабочего стола\desktop.ini
	D:\Users\feelus\YandexDisk\multimedia\vk-music\desktop.ini
	D:\Users\feelus\YandexDis

In [29]:
recycle_duplics = []
other_duplics = []
for s,n in duplics:
    files = size_root[s]
    for k in files:
        if k[0]=='multimedia':
            recycle_duplics.append(s)
            break
    else:
        other_duplics.append(s)



In [66]:
for s,n in duplics:
    files = sorted(size_root[s])
    if not all([f[0]=='multimedia' for f in files]):
        continue
    if all([not(len(f)>=2 and f[1]=='vk-music') for f in files]):
        continue
    for f in files:
        if f[1]=='vk-music':
            print('#',os.path.join(rootpath,*f))
        else:
            os.remove(os.path.join(rootpath,*f))
            print('os.remove(r"'+os.path.join(rootpath,*f)+'")')
    print()
        


# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\1516 Qazer - Viavana.mp3
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(2) Qazer\1749 Qazer - Viavana.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Weekend Heroes - Black Ops! (Original Mix) (1).mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Weekend Heroes - Black Ops! (Original Mix).mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\609 Weekend Heroes - Black Ops! (Original Mix).mp3
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\611 Weekend Heroes - Black Ops! (Original Mix) (1).mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Kila - Dusty Wine Bottle (1).mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Kila - Dusty Wine Bottle.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\641 Kila - Dusty Wine Bottle.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\audio\vk2015\Василий К. & Интеллигенты - Шантарам.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Василий К. & Интеллигенты - Шантарам.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(8) Василий К\1284 Василий К. & Интеллигенты - Шантарам.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Титаник на Фонтанке - Наутилус Помпилиус-Воздух - Воздух.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(22) Бутусов Наутилус помпилиус\1657 Титаник на Фонтанке - Наутилус Помпилиус-Воздух - Воздух.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-muz\русские\Титаник на Фонтанке - Наутилус Помпилиус-Воздух - Воздух.mp3")

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\Пикник - Недобитый романтик.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Недобитый романтик.mp3")
# D:\Users\feelus\YandexDisk\multimed

# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(84) Танец волка\057 Пикник - Много дивного на свете .mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\Пикник - Телефон.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Телефон.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(86) Иероглиф\399 Пикник - Телефон.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\Пикник - Ночь (1).mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Ночь (1).mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(82) дым\050 Пикник - Ночь.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\2e298ea60cc1.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Очень интересно.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(82) дым\047 Пик

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Дергать - МГУ.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_про учёбу\170 Дергать - МГУ.mp3
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_про учёбу\182 Дергать - МГУ (1).mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\audio\classic-rock\Track05.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(8) Королевский филармонический оркестр\1465 Королевский филармонический оркестр - Без названия (1).mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\799a1fa93dca.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Новозеландская песня.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(84) Танец волка\051 Пикник - Новозеландская песня.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\audio\classic-rock\Track06.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(8) Королевский ф

In [43]:
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Михаил Круг - Владимирский Централ (8x slow).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\440 Михаил Круг - Владимирский Централ (8x slow).mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Макс Фрай - «Чужак. Дебют в Ехо».mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\1514 Макс Фрай - «Чужак. Дебют в Ехо».mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Стивен Кинг - Стеклянный пол.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\1206 Стивен Кинг - Стеклянный пол.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Аэроглиф - Никодим-Спутник.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\и(9) Аэроглиф\1299 Аэроглиф - Никодим-Спутник.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - Cosmic Fairytale.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1583 Estas Tonne - Cosmic Fairytale.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Doors - The End.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1540 Doors - The End.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вивальди А. - The Four Seasons.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(39) Вивальди\концерты\899 Вивальди А. - The Four Seasons.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - The Winds that bring you Home.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1584 Estas Tonne - The Winds that bring you Home.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\шарль бодлер - парижский сплин (маленькие поэмы в прозе) .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\394 шарль бодлер - парижский сплин (маленькие поэмы в прозе) .mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Genesis - Dancing With The Moonlit Knight.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\801 Genesis - Dancing With The Moonlit Knight.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Most Heartful Themes Ever - Fear Not This Night (feat. Asja & Piano Version).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\space\754 Most Heartful Themes Ever - Fear Not This Night (feat. Asja & Piano Version).mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Шостакович Дмитрий Дмитриевич - Фортепианное трио №2 e-moll, Op.67 - IV. Allegre.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\1235 Шостакович Дмитрий Дмитриевич - Фортепианное трио №2 e-moll, Op.67 - IV. Allegre.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Minitronix - Feelings.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\1217 Minitronix - Feelings.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Opeth - Hessian peel.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\и(5) Opeth\1347 Opeth - Hessian peel.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\разные ! инструментальные ! which house\Astrix Vs. Pixel - Here And There.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\622 Astrix Vs. Pixel - Here And There.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Public Service Broadcasting - Tomorrow.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\1324 Public Service Broadcasting - Tomorrow.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Tomcraft & Jerome Isma-Ae - Trouble In The Redlight District (Weekend Heroes Rem.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\613 Tomcraft & Jerome Isma-Ae - Trouble In The Redlight District (Weekend Heroes Rem.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вавилонская библиотека (ТГ Варианты Возможны & ORT) - Х.Л. Борхес.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\781 Вавилонская библиотека (ТГ Варианты Возможны & ORT) - Х.Л. Борхес.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ticon & Weekend Heroes - Teleport (Original Mix).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\612 Ticon & Weekend Heroes - Teleport (Original Mix).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Карлос Кастанеда-Учение Дона Хуана - Глава 2. Часть 1.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\и(2-9) Карлос Кастанеда\653 Карлос Кастанеда-Учение Дона Хуана - Глава 2. Часть 1.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Vesilahden verajilla_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2007) Tervaskanto\247 ә_Korpiklaani_ҽ - ә_Vesilahden verajilla_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Black Sabbath - Heaven And Hell.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок\и(2) Black Sabbath\1077 Black Sabbath - Heaven And Hell.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Gilead - Herr Mannelig.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(8) Gilead\1511 Gilead - Herr Mannelig.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\НОЛЬ - На 90 градусов ниже нуля.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(9) Ноль\867 НОЛЬ - На 90 градусов ниже нуля.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\разные ! инструментальные\Arvo Pärt - Festina Lente For Strings And Harp Ad Libitum.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\614 Arvo Pärt - Festina Lente For Strings And Harp Ad Libitum.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Симфоническое Кино - Спокойная ночь.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\и(9) Симфоническое кино\1496 Симфоническое Кино - Спокойная ночь.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Mozart - Koncert fortepianowy C-dur- Allegro vivace assai.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\1059 Mozart - Koncert fortepianowy C-dur- Allegro vivace assai.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Korpiklaani - Sumussa hämärän aamun.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2012) Manala\1342 Korpiklaani - Sumussa hämärän aamun.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Surma_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2011) Ukon wacka\292 ә_Korpiklaani_ҽ - ә_Surma_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Поезд Родина - Белая Даль.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\1159 Поезд Родина - Белая Даль.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Iron Maiden - When Two Worlds Collide.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\722 Iron Maiden - When Two Worlds Collide.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Kultanainen_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2009) Karkelo\239 ә_Korpiklaani_ҽ - ә_Kultanainen_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Lacrimosa - Kaleidoskop.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1436 Lacrimosa - Kaleidoskop.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Blue Stahli - Corner.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\983 Blue Stahli - Corner.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Lera Lynn - True Detective (Curly Tonic & Franz S. Bootleg).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\и(7) Lera Lynn\1644 Lera Lynn - True Detective (Curly Tonic & Franz S. Bootleg).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Stereophonics - In A Moment.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\986 Stereophonics - In A Moment.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Metalforce - Let The Battle Begin.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок тяжелый\1065 Metalforce - Let The Battle Begin.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\The Cracow Klezmer Band - Gil Gul (The Migration Of Souls).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\164 The Cracow Klezmer Band - Gil Gul (The Migration Of Souls).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - The Song of the Golden Dragon.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1582 Estas Tonne - The Song of the Golden Dragon.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Оргия Праведников - Вперед и вверх! (Для тех, кто видит сны).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(21) Калугин Оргия праведников\678 Оргия Праведников - Вперед и вверх! (Для тех, кто видит сны).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Tung - Bullets.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\978 Tung - Bullets.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Fleur - Последний танец зимы.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\9 рок женский\и(9) Fleur\178 Fleur - Последний танец зимы.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\инструментальные ! Algambra\Algambra - The Turtle song.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(4) Algambra\1400 Algambra - The Turtle song.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Dj Маньяк и Рыбик - It's my Life.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\032 Dj Маньяк и Рыбик - It's my Life.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - Cuban Rhapsody 1917 .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1587 Estas Tonne - Cuban Rhapsody 1917.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Fleur - Баллада о белых крыльях и алых лепестках.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\9 рок женский\и(9) Fleur\1638 Fleur - Баллада о белых крыльях и алых лепестках.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ludovico Einaudi - Night.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(6) Ludovico Einaudi\1594 Ludovico Einaudi - Night.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Jay-Jay Johanson - Drowsy _ Too Young To Say Good Night.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1309 Jay-Jay Johanson - Drowsy _ Too Young To Say Good Night.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Yoko Kanno, Origa - Rise.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок женский\1213 Yoko Kanno, Origa - Rise.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вальс в Конго - Пальто (live).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\и(2) Вальс в Конго\1445 Вальс в Конго - Пальто (live).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Hengettömiltä hengiltä_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2003) Spirit of the forest\238 ә_Korpiklaani_ҽ - ә_Hengettömiltä hengiltä_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ария - Осколок льда.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\8 рок тяжелый\и(9) Кипелов - Ария\806 Ария - Осколок льда.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Caprice - Sage.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(3) Caprice\1204 Caprice - Sage.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Zodiac - Зодиак.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\мелодии\1321 Zodiac - Зодиак.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Modana ft. Carlprit - Party Crash (Extended Mix) .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\606 Modana ft. Carlprit - Party Crash (Extended Mix) .mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Wavefall - Tuty F. Fruity.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок тяжелый\1132 Wavefall - Tuty F. Fruity.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\F-777 - Space Battle.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(2) F-777\1478 F-777 - Space Battle.mp3

In [ ]:
import gc
gc.collect()

# H,I

In [73]:
rootpath_H = 'H:\\'
root_H = scan(rootpath_H)
len(root_H)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
45
46
49
52
54
55
57
58
59
60
61
62
63
64
65
66
67
74
79
85
91
92
93
94
95
96
97
98
99
100
101
102
104
111
118
119
120
121
122
123
124
125
126
127
128
129
130
131
135
139
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
160
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
184
187
191
192
193
195
196
197
198
199
200
201
202
203
204
205
206
207
208
212
213
214
215
216
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
272
274
276
278
281
282
284
285
288
289
292
293
294
296
297
298
299
300
301
303
305
306
308
309
310
311
313
315
317
318
320
321
322
323
324
325
326
327
328
330
331
332
333
335
337
338
339
340
341
344
345
347
350
353
354
356
359
362
363
365
367
368
370
371
373
376
378
379
380
382
384
386
387
389
390
391
392
393
394
39

26

In [74]:
getsize(root_H)/1024/1024

15.390410423278809

In [75]:
size_root_H={} # map размер -> множество файлов
for x,v in tree_iterator(root_H):
    size = v[0]
    file = tuple(x)
    #print(x,v,size)
    if size in size_root_H:
        size_root_H[size].add(file)
    else:
        size_root_H[size] = {file}


In [77]:
# map размер -> кол-во повторений
duplics1_H = [(i,len(size_root_H[i])) for i in reversed(sorted(size_root_H.keys())) if len(size_root_H[i])!=1]
tot_s=0
tot_ss = 0
tot_n = 0
for s,n in duplics1_H:
    tot_s+=s*(n-1)
    tot_ss+=s*(n)
    tot_n+=n
print('размер всего, размер лишнего, кол-во разных, кол-во всего')
print('%.6f'%(tot_ss/1024/1024),'%.6f'%(tot_s/1024/1024),len(duplics1_H),tot_n)

размер всего, размер лишнего, кол-во разных, кол-во всего
81706.756777 50592.305844 40054 424103


In [78]:
for s,n in duplics1_H[1:100]:
    print('%.3f'%(s/1024))
    for q in sorted(size_root_H[s]):
        print('\t'+os.path.join(rootpath_H,*q))
        


1529296.875
	H:\Mathematica_11.1.0_WIN_official\setup-1b.bin
	H:\backup\Downloads\Ил-2 Штурмовик Битва за Британию (2011)\IL.2.Sturmovik.Cliffs.of.Dover.RU-R.G.Игроманы.exe
	H:\backup\Downloads\Ил-2 Штурмовик Битва за Британию (2011)\IL.2.Sturmovik.Cliffs.of.Dover.RU-R.G.Игроманы.r00
	H:\backup\Downloads\Ил-2 Штурмовик Битва за Британию (2011)\IL.2.Sturmovik.Cliffs.of.Dover.RU-R.G.Игроманы.r01
1038669.739
	H:\yadisk-backup\notsynch_plan\temp_all\Установщики\Heroes 3 Complete , Chronicles , Wog + Era\HoMaM_III_Complete_RePack\setup-1.bin
	H:\yadisk-backup\программы-установщики\Установщики\Heroes 3 Complete , Chronicles , Wog + Era\HoMaM_III_Complete_RePack\setup-1.bin
835834.286
	H:\yadisk-arxiv\Программы-установщики\MS_Office_2007_RePack_V12.5.exe
	H:\yadisk-backup\notsynch_plan\MS_Office_2007_RePack_V12.5.exe
	H:\yadisk-backup\программы-установщики\MS_Office_2007_RePack_V12.5.exe
730816.000
	H:\yadisk-arxiv\Программы-установщики\PROMT PRO 9.5.ISO
	H:\yadisk-backup\программы-установщик

In [62]:
tot_ss = 0
tot_n = 0
for s,n in duplics1:
    for p in sorted(size_root[s]):
        hashh = md5(os.path.join(rootpath,*p))
        get_subtree(root,p).append(hashh)
        #print(hashh+'\t'+os.path.join(rootpath,*p))
        tot_ss += s
        tot_n+=1
        print(tot_n,tot_ss/1024/1024)

1 48.181640625
2 96.36328125
3 134.974609375
4 173.5859375
5 202.80859375
6 232.03125
7 250.927734375
8 269.82421875
9 286.328125
10 302.83203125
11 314.35618782043457
12 325.88034439086914
13 335.54831314086914
14 345.21628189086914
15 354.6856441497803
16 364.1550064086914
17 373.5612564086914
18 382.9675064086914
19 391.0983657836914
20 399.2292251586914
21 406.80241298675537
22 414.37560081481934
23 421.87854766845703
24 429.3814945220947
25 436.8072757720947
26 444.2330570220947
27 451.0649356842041
28 457.8968143463135
29 464.72869300842285
30 471.5605716705322
31 478.3904972076416
32 485.220422744751
33 492.05034828186035
34 498.8802738189697
35 505.57690048217773
36 512.2735271453857
37 518.8789958953857
38 525.4844646453857
39 531.7930583953857
40 538.1016521453857
41 544.2901248931885
42 550.4785976409912
43 556.5567226409912
44 562.6348476409912
45 568.7012538909912
46 574.7676601409912
47 580.7690353393555
48 586.7704105377197
49 592.5939111709595
50 598.4174118041992
51 60

379 1312.8068981170654
380 1313.5764293670654
381 1314.3459606170654
382 1315.1154918670654
383 1315.883264541626
384 1316.6510372161865
385 1317.41219997406
386 1318.1733627319336
387 1318.9314489364624
388 1319.6895351409912
389 1320.4435329437256
390 1321.19753074646
391 1321.939769744873
392 1322.6820087432861
393 1323.4209804534912
394 1324.1599521636963
395 1324.8954391479492
396 1325.6309261322021
397 1326.3620176315308
398 1327.0931091308594
399 1327.8234272003174
400 1328.5537452697754
401 1329.2840633392334
402 1330.0143814086914
403 1330.7446994781494
404 1331.4750175476074
405 1332.1976737976074
406 1332.9203300476074
407 1333.6429862976074
408 1334.3607807159424
409 1335.0785751342773
410 1335.795615196228
411 1336.5126552581787
412 1337.2182235717773
413 1337.923791885376
414 1338.628870010376
415 1339.333948135376
416 1340.039026260376
417 1340.744104385376
418 1341.445634841919
419 1342.147165298462
420 1342.8362588882446
421 1343.5253524780273
422 1344.2054281234741
42

740 1514.6771030426025
741 1515.0706577301025
742 1515.4642124176025
743 1515.8577671051025
744 1516.2513217926025
745 1516.6370639801025
746 1517.0228061676025
747 1517.4085483551025
748 1517.7942905426025
749 1518.1800327301025
750 1518.5657749176025
751 1518.948182106018
752 1519.3305892944336
753 1519.7119369506836
754 1520.0932846069336
755 1520.4737424850464
756 1520.8542003631592
757 1521.2331066131592
758 1521.6120128631592
759 1521.9909191131592
760 1522.3698253631592
761 1522.7482061386108
762 1523.1265869140625
763 1523.5045166015625
764 1523.8824462890625
765 1524.2603759765625
766 1524.637511253357
767 1525.0146465301514
768 1525.3904933929443
769 1525.7663402557373
770 1526.1410474777222
771 1526.515754699707
772 1526.886848449707
773 1527.257942199707
774 1527.629035949707
775 1528.000129699707
776 1528.3583755493164
777 1528.7166213989258
778 1529.0747365951538
779 1529.4328517913818
780 1529.7905282974243
781 1530.1482048034668
782 1530.5058813095093
783 1530.863557815

1100 1622.6537284851074
1101 1622.8983573913574
1102 1623.1419553756714
1103 1623.3855533599854
1104 1623.6278190612793
1105 1623.8700847625732
1106 1624.1123180389404
1107 1624.3545513153076
1108 1624.5957622528076
1109 1624.8369731903076
1110 1625.0774154663086
1111 1625.3178577423096
1112 1625.558162689209
1113 1625.7984676361084
1114 1626.037878036499
1115 1626.2772884368896
1116 1626.5165462493896
1117 1626.7558040618896
1118 1626.9946460723877
1119 1627.2334880828857
1120 1627.4723300933838
1121 1627.7111721038818
1122 1627.949951171875
1123 1628.1887302398682
1124 1628.4275093078613
1125 1628.6662883758545
1126 1628.9050674438477
1127 1629.1438465118408
1128 1629.3821277618408
1129 1629.6204090118408
1130 1629.8578281402588
1131 1630.0952472686768
1132 1630.3325519561768
1133 1630.5698566436768
1134 1630.8071613311768
1135 1631.0444660186768
1136 1631.2817707061768
1137 1631.5190753936768
1138 1631.7563800811768
1139 1631.9936847686768
1140 1632.2309894561768
1141 1632.468294143

1449 1695.1923427581787
1450 1695.3701124191284
1451 1695.5478820800781
1452 1695.7242250442505
1453 1695.9005680084229
1454 1696.0763492584229
1455 1696.2521305084229
1456 1696.4279117584229
1457 1696.6019268035889
1458 1696.7759418487549
1459 1696.9490957260132
1460 1697.1222496032715
1461 1697.2946128845215
1462 1697.4669761657715
1463 1697.6391897201538
1464 1697.8114032745361
1465 1697.9836168289185
1466 1698.1558303833008
1467 1698.3277053833008
1468 1698.4995803833008
1469 1698.6714553833008
1470 1698.8433303833008
1471 1699.0152053833008
1472 1699.1870803833008
1473 1699.3589553833008
1474 1699.5308303833008
1475 1699.7027053833008
1476 1699.8745622634888
1477 1700.0464191436768
1478 1700.2182760238647
1479 1700.3901329040527
1480 1700.5619897842407
1481 1700.7338466644287
1482 1700.9057035446167
1483 1701.0775604248047
1484 1701.2494173049927
1485 1701.4212741851807
1486 1701.5926609039307
1487 1701.7640476226807
1488 1701.9351863861084
1489 1702.1063251495361
1490 1702.277365

...

44224 2158.805467605591
44225 2158.805528640747
44226 2158.8055896759033
44227 2158.8056507110596
44228 2158.805711746216
44229 2158.805772781372
44230 2158.8058338165283
44231 2158.8058948516846
44232 2158.805955886841
44233 2158.806016921997
44234 2158.8060779571533
44235 2158.8061389923096
44236 2158.806200027466
44237 2158.806261062622
44238 2158.8063220977783
44239 2158.8063831329346
44240 2158.806444168091
44241 2158.806505203247
44242 2158.8065662384033
44243 2158.8066272735596
44244 2158.806688308716
44245 2158.806749343872
44246 2158.8068103790283
44247 2158.8068714141846
44248 2158.806932449341
44249 2158.806993484497
44250 2158.8070545196533
44251 2158.8071155548096
44252 2158.807176589966
44253 2158.807237625122
44254 2158.8072986602783
44255 2158.8073596954346
44256 2158.807420730591
44257 2158.807481765747
44258 2158.8075428009033
44259 2158.8076038360596
44260 2158.807664871216
44261 2158.807725906372
44262 2158.8077869415283
44263 2158.8078479766846
44264 2158.807909011

44704 2158.832769393921
44705 2158.8328199386597
44706 2158.8328704833984
44707 2158.832921028137
44708 2158.8329706192017
44709 2158.833020210266
44710 2158.8330698013306
44711 2158.833119392395
44712 2158.8331689834595
44713 2158.833218574524
44714 2158.8332681655884
44715 2158.833317756653
44716 2158.8333673477173
44717 2158.8334159851074
44718 2158.8334646224976
44719 2158.8335132598877
44720 2158.833561897278
44721 2158.833610534668
44722 2158.833659172058
44723 2158.8337078094482
44724 2158.8337564468384
44725 2158.8338050842285
44726 2158.8338537216187
44727 2158.833902359009
44728 2158.833950996399
44729 2158.833999633789
44730 2158.834048271179
44731 2158.8340969085693
44732 2158.8341455459595
44733 2158.8341941833496
44734 2158.8342428207397
44735 2158.83429145813
44736 2158.83434009552
44737 2158.83438873291
44738 2158.8344373703003
44739 2158.8344860076904
44740 2158.8345346450806
44741 2158.8345832824707
44742 2158.834631919861
44743 2158.834680557251
44744 2158.8347291946

45183 2158.85281085968
45184 2158.8528480529785
45185 2158.8528842926025
45186 2158.8529205322266
45187 2158.8529567718506
45188 2158.8529930114746
45189 2158.8530292510986
45190 2158.8530654907227
45191 2158.8531007766724
45192 2158.853136062622
45193 2158.853171348572
45194 2158.8532066345215
45195 2158.853240966797
45196 2158.8532752990723
45197 2158.8533096313477
45198 2158.853343963623
45199 2158.8533782958984
45200 2158.853412628174
45201 2158.853446960449
45202 2158.8534812927246
45203 2158.853515625
45204 2158.853549003601
45205 2158.853582382202
45206 2158.853615760803
45207 2158.8536491394043
45208 2158.8536825180054
45209 2158.8537158966064
45210 2158.8537492752075
45211 2158.8537826538086
45212 2158.8538160324097
45213 2158.8538494110107
45214 2158.853882789612
45215 2158.853916168213
45216 2158.853949546814
45217 2158.853982925415
45218 2158.854016304016
45219 2158.854049682617
45220 2158.8540830612183
45221 2158.8541164398193
45222 2158.8541498184204
45223 2158.8541822433

45648 2158.863121986389
45649 2158.8631343841553
45650 2158.8631467819214
45651 2158.8631591796875
45652 2158.8631715774536
45653 2158.8631839752197
45654 2158.863196372986
45655 2158.863208770752
45656 2158.8632202148438
45657 2158.8632316589355
45658 2158.8632431030273
45659 2158.863254547119
45660 2158.863265991211
45661 2158.8632774353027
45662 2158.8632888793945
45663 2158.8633003234863
45664 2158.863311767578
45665 2158.86332321167
45666 2158.8633346557617
45667 2158.863345146179
45668 2158.8633556365967
45669 2158.863366127014
45670 2158.8633766174316
45671 2158.863387107849
45672 2158.8633975982666
45673 2158.863408088684
45674 2158.8634176254272
45675 2158.8634271621704
45676 2158.8634366989136
45677 2158.8634462356567
45678 2158.8634557724
45679 2158.863465309143
45680 2158.8634748458862
45681 2158.8634843826294
45682 2158.8634939193726
45683 2158.8635034561157
45684 2158.863512992859
45685 2158.863522529602
45686 2158.863532066345
45687 2158.8635416030884
45688 2158.86355113

46097 2158.8644218444824
46098 2158.8644218444824
46099 2158.8644218444824
46100 2158.8644218444824
46101 2158.8644218444824
46102 2158.8644218444824
46103 2158.8644218444824
46104 2158.8644218444824
46105 2158.8644218444824
46106 2158.8644218444824
46107 2158.8644218444824
46108 2158.8644218444824
46109 2158.8644218444824
46110 2158.8644218444824
46111 2158.8644218444824
46112 2158.8644218444824
46113 2158.8644218444824
46114 2158.8644218444824
46115 2158.8644218444824
46116 2158.8644218444824
46117 2158.8644218444824
46118 2158.8644218444824
46119 2158.8644218444824
46120 2158.8644218444824
46121 2158.8644218444824
46122 2158.8644218444824
46123 2158.8644218444824
46124 2158.8644218444824
46125 2158.8644218444824
46126 2158.8644218444824
46127 2158.8644218444824


In [63]:
duplics2 = {}
for s,n in duplics1:
    for p in sorted(size_root[s]):
        fdata = get_subtree(root,p)
        assert len(fdata)>=2, (p,fdata)
        if len(fdata)>2: del fd[2:]
        hashh = fdata[1]
        if (s,hashh) in duplics2:
            duplics2[(s,hashh)].add(p)
        else:
            duplics2[(s,hashh)] = {p}
duplics2 = {s_h:pp for (s_h,pp) in duplics2.items() if len(pp)>1}
tot_s=0
tot_ss = 0
tot_n = 0
for s_h,pp in duplics2.items():
    n = len(pp)
    s = s_h[0]
    tot_s+=s*(n-1)
    tot_ss+=s*(n)
    tot_n+=n
print('размер всего, размер лишнего, кол-во разных, кол-во всего')
print(tot_ss/1024/1024,tot_s/1024/1024,len(duplics2),tot_n)

размер всего, размер лишнего, кол-во разных, кол-во всего
1319.6228761672974 735.7141628265381 9454 28307


In [65]:
for s_h,pp in [(a,b) for a,b in duplics2.items()]:
    s,h = s_h
    for p in pp:
        if p[0]=='_programming' and p[1]=='_lang_cpp' :
            print('%.3f'%(s/1024),h)
            for q in pp:
                print('\t'+os.path.join(rootpath,*q))
            break

6336.996 3e5c94b472687d947afd0721390e3499
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\lib_QT\progs\instal_log.txt
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\bin\QT_progs\instal_log.txt
3131.000 d41029d0ccb07fda0f585b3969ca9e93
	D:\Users\feelus\YandexDisk\_programming_arxiv\src_2013\strlib\strlib.ncb
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\strlib\strlib.ncb
1573.876 45b3548bde0815a6896b459cda131637
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\CPP_Korukhova.pdf
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\[Корухова] Сборник задач и управжнений по с++.pdf
1508.636 c2f2e4610ac48aa42e0f61091d348a98
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\bin\BC_3.1\BC\TCHELP.TCH
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\приложения\BC_3.1\BC\TCHELP.TCH
1219.000 bf59307d71b39ee7bea74f07aa11c820
	D:\Users\feelus\YandexDisk\_programming_arxiv\src_2013\strlib\Debug\strlib.pdb
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\strlib\Debug\strlib.p

	D:\Users\feelus\YandexDisk\_programming_arxiv\lang_delphy_pascal\lang_pascal\compilators\TP_ru-board\TP\BGI\SIMP.CHR
	D:\Users\feelus\YandexDisk\_programming_arxiv\lang_delphy_pascal\lang_pascal\compilators\BPdos\BGI\SIMP.CHR
	D:\Users\feelus\YandexDisk\_programming_arxiv\lang_delphy_pascal\паскаль\compilators\BPwin\BGI\SIMP.CHR
	D:\Users\feelus\YandexDisk\_programming_arxiv\lang_delphy_pascal\паскаль\compilators\TP_ru-board\TP\BGI\SIMP.CHR
	D:\Users\feelus\YandexDisk\_programming_arxiv\lang_delphy_pascal\паскаль\compilators\BPdos\BGI\SIMP.CHR
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\bin\BC_3.1\BC\BGI\SIMP.CHR
8.168 b3fb3996e7907131db05e571f38bbfc6
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\bin\pugixml-1.0\scripts\pugixml_vs2010.vcxproj
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\приложения\pugixml-1.0\scripts\pugixml_vs2010.vcxproj
8.000 bcbd6ce928235b7bc473d61dd73db836
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\приложения\BC_3.1\BC\Include\ST

	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\грамматика с++\грамматика с++\dot\Node, Edge and Graph Attributes_files\g_rad90.png
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\описание данных\Node, Edge and Graph Attributes_files\g_rad90.png
1.555 7aa136ebb7907809dab986de93485f73
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\грамматика с++\грамматика с++\dot\Node, Edge and Graph Attributes_files\g_rad180.png
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\описание данных\Node, Edge and Graph Attributes_files\g_rad180.png
1.552 d193f202fb9f43e1560a39f6094cbb4f
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\приложения\BC_3.1\BC\Include\GENERIC.H
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\bin\BC_3.1\BC\Include\GENERIC.H
1.547 1508d78001f23135bb27271666b0b132
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\приложения\BC_3.1\BC\Include\STDIOSTR.H
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\bin\BC_3.1\BC\Include\STDIOSTR.H
1.544 cff2cbf

	D:\Users\feelus\YandexDisk\_programming_arxiv\src_2000\progs\myIO\список файлов\files_2012_3_13_19_58_58_.txt
	D:\Users\feelus\YandexDisk\_programming_arxiv\src_2000\research\preprocessor2\prepro\prepro\first.h
	D:\Users\feelus\YandexDisk\_programming_arxiv\src_2000\progs\старое\learnlibr\vc\LearnLibr\LearnLibr\Debug\LearnLibr.unsuccessfulbuild
	D:\Users\feelus\YandexDisk\Documents\список файлов\files_2012_3_11_18_27_34_.txt
	D:\Users\feelus\YandexDisk\_programming\old-pascal-projekts\FRE\К НОСОВА\0.txt
	D:\Users\feelus\YandexDisk\_desktop\перевод\Звезды на Земле  Термояд — Популярная механика_files\rle.cgi
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\lib_glib-2.32.3\gio\tests\nothing.pem
	D:\Users\feelus\YandexDisk\_programming\old-pascal-projekts\FRE\А ЛУШИН\0.txt
	D:\Users\feelus\YandexDisk\_programming_arxiv\lang_tex\latex\безназвания-1.log
	D:\Users\feelus\YandexDisk\описание данных\notsynch_all\установщики\дрова для планшета\Camera.txt
	D:\Users\feelus\YandexDisk\описание 

In [36]:
for s_h,pp in [(a,b) for a,b in duplics2.items()]:
    s,h = s_h
    for p in pp:
        if p[0]=='multimedia':
            print(int(s),h)
            for p in pp:
                print('\t'+os.path.join(rootpath,*p))
            break

7569 0a025414a5f4cfc778f8a52ef5d2ffb7
	D:\Users\feelus\YandexDisk\multimedia\.files\2017-07-08 16-30-48-dirs
	D:\Users\feelus\YandexDisk\multimedia\.files\2017-07-08 16-27-57-dirs
325 21e67c73585028975d7ab33cc808ed94
	D:\Users\feelus\YandexDisk\multimedia\статы - Видео Шведова\content-link.txt
	D:\Users\feelus\YandexDisk\_физика\5_статистическая физика\content-link.txt
212 cb0de00f716742da5d504fe8e181062e
	D:\Users\feelus\YandexDisk\окюу3\desktop.ini
	D:\Users\feelus\YandexDisk\multimedia\audio\desktop.ini
	D:\Users\feelus\YandexDisk\_физика\3_квантовые вычисления\desktop.ini
	D:\Users\feelus\YandexDisk\_programming_arxiv\lang_asm\desktop.ini
	D:\Users\feelus\YandexDisk\multimedia\фото - 409-м\desktop.ini
	D:\Users\feelus\YandexDisk\программы-установщики\_крякнутые\ABBYY Screenshot Reader 11.0.113.201 RePack by KpoJIuK\desktop.ini
	D:\Users\feelus\YandexDisk\multimedia\обои рабочего стола\desktop.ini
	D:\Users\feelus\YandexDisk\multimedia\vk-music\desktop.ini
	D:\Users\feelus\YandexDis

In [29]:
recycle_duplics = []
other_duplics = []
for s,n in duplics:
    files = size_root[s]
    for k in files:
        if k[0]=='multimedia':
            recycle_duplics.append(s)
            break
    else:
        other_duplics.append(s)



In [43]:
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Михаил Круг - Владимирский Централ (8x slow).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\440 Михаил Круг - Владимирский Централ (8x slow).mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Макс Фрай - «Чужак. Дебют в Ехо».mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\1514 Макс Фрай - «Чужак. Дебют в Ехо».mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Стивен Кинг - Стеклянный пол.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\1206 Стивен Кинг - Стеклянный пол.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Аэроглиф - Никодим-Спутник.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\и(9) Аэроглиф\1299 Аэроглиф - Никодим-Спутник.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - Cosmic Fairytale.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1583 Estas Tonne - Cosmic Fairytale.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Doors - The End.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1540 Doors - The End.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вивальди А. - The Four Seasons.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(39) Вивальди\концерты\899 Вивальди А. - The Four Seasons.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - The Winds that bring you Home.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1584 Estas Tonne - The Winds that bring you Home.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\шарль бодлер - парижский сплин (маленькие поэмы в прозе) .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\394 шарль бодлер - парижский сплин (маленькие поэмы в прозе) .mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Genesis - Dancing With The Moonlit Knight.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\801 Genesis - Dancing With The Moonlit Knight.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Most Heartful Themes Ever - Fear Not This Night (feat. Asja & Piano Version).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\space\754 Most Heartful Themes Ever - Fear Not This Night (feat. Asja & Piano Version).mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Шостакович Дмитрий Дмитриевич - Фортепианное трио №2 e-moll, Op.67 - IV. Allegre.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\1235 Шостакович Дмитрий Дмитриевич - Фортепианное трио №2 e-moll, Op.67 - IV. Allegre.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Minitronix - Feelings.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\1217 Minitronix - Feelings.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Opeth - Hessian peel.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\и(5) Opeth\1347 Opeth - Hessian peel.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\разные ! инструментальные ! which house\Astrix Vs. Pixel - Here And There.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\622 Astrix Vs. Pixel - Here And There.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Public Service Broadcasting - Tomorrow.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\1324 Public Service Broadcasting - Tomorrow.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Tomcraft & Jerome Isma-Ae - Trouble In The Redlight District (Weekend Heroes Rem.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\613 Tomcraft & Jerome Isma-Ae - Trouble In The Redlight District (Weekend Heroes Rem.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вавилонская библиотека (ТГ Варианты Возможны & ORT) - Х.Л. Борхес.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\781 Вавилонская библиотека (ТГ Варианты Возможны & ORT) - Х.Л. Борхес.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ticon & Weekend Heroes - Teleport (Original Mix).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\612 Ticon & Weekend Heroes - Teleport (Original Mix).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Карлос Кастанеда-Учение Дона Хуана - Глава 2. Часть 1.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\и(2-9) Карлос Кастанеда\653 Карлос Кастанеда-Учение Дона Хуана - Глава 2. Часть 1.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Vesilahden verajilla_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2007) Tervaskanto\247 ә_Korpiklaani_ҽ - ә_Vesilahden verajilla_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Black Sabbath - Heaven And Hell.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок\и(2) Black Sabbath\1077 Black Sabbath - Heaven And Hell.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Gilead - Herr Mannelig.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(8) Gilead\1511 Gilead - Herr Mannelig.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\НОЛЬ - На 90 градусов ниже нуля.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(9) Ноль\867 НОЛЬ - На 90 градусов ниже нуля.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\разные ! инструментальные\Arvo Pärt - Festina Lente For Strings And Harp Ad Libitum.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\614 Arvo Pärt - Festina Lente For Strings And Harp Ad Libitum.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Симфоническое Кино - Спокойная ночь.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\и(9) Симфоническое кино\1496 Симфоническое Кино - Спокойная ночь.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Mozart - Koncert fortepianowy C-dur- Allegro vivace assai.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\1059 Mozart - Koncert fortepianowy C-dur- Allegro vivace assai.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Korpiklaani - Sumussa hämärän aamun.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2012) Manala\1342 Korpiklaani - Sumussa hämärän aamun.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Surma_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2011) Ukon wacka\292 ә_Korpiklaani_ҽ - ә_Surma_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Поезд Родина - Белая Даль.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\1159 Поезд Родина - Белая Даль.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Iron Maiden - When Two Worlds Collide.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\722 Iron Maiden - When Two Worlds Collide.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Kultanainen_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2009) Karkelo\239 ә_Korpiklaani_ҽ - ә_Kultanainen_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Lacrimosa - Kaleidoskop.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1436 Lacrimosa - Kaleidoskop.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Blue Stahli - Corner.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\983 Blue Stahli - Corner.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Lera Lynn - True Detective (Curly Tonic & Franz S. Bootleg).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\и(7) Lera Lynn\1644 Lera Lynn - True Detective (Curly Tonic & Franz S. Bootleg).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Stereophonics - In A Moment.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\986 Stereophonics - In A Moment.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Metalforce - Let The Battle Begin.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок тяжелый\1065 Metalforce - Let The Battle Begin.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\The Cracow Klezmer Band - Gil Gul (The Migration Of Souls).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\164 The Cracow Klezmer Band - Gil Gul (The Migration Of Souls).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - The Song of the Golden Dragon.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1582 Estas Tonne - The Song of the Golden Dragon.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Оргия Праведников - Вперед и вверх! (Для тех, кто видит сны).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(21) Калугин Оргия праведников\678 Оргия Праведников - Вперед и вверх! (Для тех, кто видит сны).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Tung - Bullets.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\978 Tung - Bullets.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Fleur - Последний танец зимы.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\9 рок женский\и(9) Fleur\178 Fleur - Последний танец зимы.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\инструментальные ! Algambra\Algambra - The Turtle song.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(4) Algambra\1400 Algambra - The Turtle song.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Dj Маньяк и Рыбик - It's my Life.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\032 Dj Маньяк и Рыбик - It's my Life.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - Cuban Rhapsody 1917 .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1587 Estas Tonne - Cuban Rhapsody 1917.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Fleur - Баллада о белых крыльях и алых лепестках.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\9 рок женский\и(9) Fleur\1638 Fleur - Баллада о белых крыльях и алых лепестках.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ludovico Einaudi - Night.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(6) Ludovico Einaudi\1594 Ludovico Einaudi - Night.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Jay-Jay Johanson - Drowsy _ Too Young To Say Good Night.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1309 Jay-Jay Johanson - Drowsy _ Too Young To Say Good Night.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Yoko Kanno, Origa - Rise.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок женский\1213 Yoko Kanno, Origa - Rise.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вальс в Конго - Пальто (live).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\и(2) Вальс в Конго\1445 Вальс в Конго - Пальто (live).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Hengettömiltä hengiltä_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2003) Spirit of the forest\238 ә_Korpiklaani_ҽ - ә_Hengettömiltä hengiltä_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ария - Осколок льда.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\8 рок тяжелый\и(9) Кипелов - Ария\806 Ария - Осколок льда.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Caprice - Sage.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(3) Caprice\1204 Caprice - Sage.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Zodiac - Зодиак.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\мелодии\1321 Zodiac - Зодиак.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Modana ft. Carlprit - Party Crash (Extended Mix) .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\606 Modana ft. Carlprit - Party Crash (Extended Mix) .mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Wavefall - Tuty F. Fruity.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок тяжелый\1132 Wavefall - Tuty F. Fruity.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\F-777 - Space Battle.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(2) F-777\1478 F-777 - Space Battle.mp3

In [ ]:
import gc
gc.collect()